In [1]:
import os
os.chdir('D:\\DA\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'Data/Variants/SIR/omicron/*.csv'
dic_files = Load_files.load_files(path, -14, -12)
data = dic_files['대구']
data

,stdDay,Susceptible,22D_Infected,22D_Recovered,22D_Dead,22D_alpha,22D_beta,22D_gamma,23A_Infected,23A_Recovered,23A_Dead,23A_alpha,23A_beta,23A_gamma
0,2022-12-19,1154500,7646.89,28392.09,32.73,0.470884,0.191027,0.000145,9.73,0.00,0.00,7.709827,0.000000,0.000000
1,2022-12-20,1150875,7925.02,29852.85,33.84,0.454658,0.172557,0.000233,45.98,0.00,0.00,1.632581,0.000000,0.000000
2,2022-12-21,1147259,8291.33,31220.37,35.69,0.385427,0.137623,0.000134,82.14,0.00,0.00,0.810531,0.000000,0.000000
3,2022-12-22,1144062,8683.70,32361.45,36.80,0.306706,0.122074,0.000170,114.11,0.00,0.00,0.486252,0.000000,0.000000
4,2022-12-23,1141405,8897.53,33421.50,38.28,0.319341,0.127748,0.000125,140.68,0.00,0.00,0.420776,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,2023-04-30,1031603,503.31,104434.19,135.79,0.106662,0.063420,0.000000,1695.36,9178.20,10.14,0.106662,0.063420,0.000000
133,2023-05-01,1031481,494.57,104466.11,135.79,0.427119,0.164426,0.000384,1665.92,9285.72,10.14,0.427119,0.164426,0.000384
134,2023-05-02,1031001,504.26,104547.43,135.98,0.405137,0.161266,0.000377,1698.56,9559.64,10.78,0.405137,0.161266,0.000377
135,2023-05-03,1030537,510.91,104628.75,136.17,0.505227,0.147267,0.000000,1720.96,9833.56,11.42,0.505227,0.147267,0.000000


In [3]:
learning_rates_list = [1e-3, 1e-4, 1e-5]
patiences_list = [20, 50]
num_layers_list = [1, 2, 4, 8]
batch_sizes_list = [32, 64]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25]

hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for batch_sizes in batch_sizes_list:
                for hidden_size in hidden_sizes_list:    
                    for dropout in dropout_list:
                        hyperparameter_dict[i] = [lr, patience, num_layers, batch_sizes, hidden_size, dropout]
                        i += 1
                        
print(i)

144


In [4]:
criterion = nn.MSELoss()

input_size = 3
sequence_length = 60
num_epochs = 10000

df = Prepare_df.processing(data, 'stdDay', '23A_alpha')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

MinMaxScaler()

In [5]:
best_model = {}
models_list = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU']

for num_model in range(6):
    rmse_min = np.inf
    for hyperparameter in tqdm(hyperparameter_dict.values(), desc = f'대구_{models_list[num_model]}', position = 0):
        
        lr = hyperparameter[0]
        patience = hyperparameter[1]
        num_layers = hyperparameter[2]
        batch_size = hyperparameter[3]
        hidden_size = hyperparameter[4]
        dropout = hyperparameter[5]
        #print('-------------------------------------------------------------------------------------')
        #print('lr:', lr, ' patience:', patience, ' num_layers:', num_layers, 
        #      ' batch_size:', batch_size, ' hidden_size:', hidden_size, ' dropout:', dropout)
    
        x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 61, 60, 1, batch_size, 'mto')
        
        if num_model == 0:
            model = RNN(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)
            
        elif num_model == 1:
            model = LSTM(input_size = input_size,
                         hidden_size = hidden_size,
                         sequence_length = sequence_length,
                         num_layers = num_layers, 
                         dropout = dropout, 
                         device = device).to(device)
        
        elif num_model == 2:
            model = GRU(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)
            
        elif num_model == 3:
            model = BiRNN(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        elif num_model == 4:
            model = BiLSTM(input_size = input_size,
                           hidden_size = hidden_size,
                           sequence_length = sequence_length,
                           num_layers = num_layers, 
                           dropout = dropout, 
                           device = device).to(device)

        elif num_model == 5:
            model = BiGRU(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        optimizer = Adam(model.parameters(), lr = lr)
    
        loss_list, model, epoch = Trainer.Many_to_One(train_loader, 
                                                      test_loader, 
                                                      model, 
                                                      criterion, 
                                                      optimizer, 
                                                      num_epochs, 
                                                      patience,
                                                      device)
    
        label_y, predicted = predict_mto(model, df, x_ss, y_ms)
    
        if rmse_min > rmse(label_y[-7:], predicted[-7:]):
            rmse_min = rmse(label_y[-7:], predicted[-7:])
            best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                  num_layers, batch_size, hidden_size, dropout]

대구_RNN:   0%|          | 0/144 [00:00<?, ?it/s]

대구_LSTM:   0%|          | 0/144 [00:00<?, ?it/s]

대구_GRU:   0%|          | 0/144 [00:00<?, ?it/s]

대구_BiRNN:   0%|          | 0/144 [00:00<?, ?it/s]

대구_BiLSTM:   0%|          | 0/144 [00:00<?, ?it/s]

대구_BiGRU:   0%|          | 0/144 [00:00<?, ?it/s]

In [6]:
best_model

{'RNN': [0.12103324193642515,
  RNN(
    (rnn): RNN(3, 8, batch_first=True, dropout=0.25)
    (fc1): Linear(in_features=480, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=1, bias=True)
    (relu): ReLU()
  ),
  [0.31496621668338776,
   0.2977824807167053,
   0.28188464045524597,
   0.2667496055364609,
   0.2522892504930496,
   0.23870573192834854,
   0.22566545754671097,
   0.21290040016174316,
   0.20036092400550842,
   0.1878923922777176,
   0.17548247426748276,
   0.162903293967247,
   0.15021956712007523,
   0.13752521574497223,
   0.12487092614173889,
   0.1123148761689663,
   0.10006418079137802,
   0.08819722384214401,
   0.07685194164514542,
   0.06617129221558571,
   0.056310342624783516,
   0.04738956317305565,
   0.03950396925210953,
   0.03277269937098026,
   0.027256098575890064,
   0.02294307667762041,
   0.019777503330260515,
   0.017627783585339785,
   0.01628902042284608,
   0.

In [8]:
for key, values in best_model.items():
    save_model(values[1].state_dict(), f"model/SIR/23A/{key}.pth")
    save_hyperparameter(values[3:], f"hyperparameter/SIR/23A/{key}.pkl")